# Function Calling

Code inspired by [Openai Cookbook](https://cookbook.openai.com/examples/how_to_call_functions_with_chat_models).

In [64]:
from dotenv import load_dotenv
load_dotenv();
import json
from openai import OpenAI
from formatting import pprint_messages
GPT_MODEL = "gpt-4o"
client = OpenAI()

First let's define a way for making calls to the Chat Completions API.

In [47]:
def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

## Basics Concepts

In June 2023, [OpenAI announced](https://openai.com/index/function-calling-and-other-api-updates/) that they were going to add support for “function calling capabilities” in their API. **Function calling** has since become a common interface to the LLM where the user provides a list of one or more “functions” and accompanying arguments for that function. The model can then choose to call a function and provide the necessary arguments.

Here is an example from the evaluation set showing how a function calling works. Here is a function called  `get_current_weather`:

```python
"function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                },
                "required": ["location", "format"],
            },
        }
```

 As we can see the model now has access to a function it can “call” (really it just returns the function name and argument values, nothing ‘happens’ during function calling).

The function, `get_current_weather` has two required arguments: `location` and `format`.

When prompted with a question such as:

> "Find the area of a triangle with a base of 10 units and height of 5 units."

The model can “call” the function by replying with a response like this: 

```python
[calculate_triangle_area(base=10, height=5)]
```

The main benefit of function calling is that we have a reliable method for having an LLM communicate with other systems and provide a formatted response. It’s easy to image this code being transformed into an actual function call in a language like Python or turned into a request to an API endpoint. 


In [48]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                },
                "required": ["location", "format"],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_n_day_weather_forecast",
            "description": "Get an N-day weather forecast",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                    "num_days": {
                        "type": "integer",
                        "description": "The number of days to forecast",
                    }
                },
                "required": ["location", "format", "num_days"]
            },
        }
    },
]

In [49]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "What's the weather like today"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append({"role": "assistant", "content": assistant_message.content})
pprint_messages(messages)

╭─ System ─────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Don't make assumptions about what values to plug into functions. Ask for clarification if a     │
│  user request is ambiguous.                                                                      │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ User ───────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  What's the weather like today                                                                   │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Assistant ──────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Could you please provide the location for which you would like to know the weather?             │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

In [50]:
messages.append({"role": "user", "content": "I'm in Santiago, Chile."})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
if len(assistant_message.tool_calls) > 0:
    messages.append({"role": "function", "content": f"{assistant_message.tool_calls[0].function}"})

pprint_messages(messages)

╭─ System ─────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Don't make assumptions about what values to plug into functions. Ask for clarification if a     │
│  user request is ambiguous.                                                                      │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ User ───────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  What's the weather like today                                                                   │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Assistant ──────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Could you please provide the location for which you would like to know the weather?             │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ User ───────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  I'm in Santiago, Chile.                                                                         │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Function ───────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Function(arguments='{"location":"Santiago, Chile","format":"celsius"}',                         │
│  name='get_current_weather')                                                                     │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

In [51]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "What is the weather going to be like in Santiago, Chile over the next x days?"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append({"role": "assistant", "content": assistant_message.content})
pprint_messages(messages)


╭─ System ─────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Don't make assumptions about what values to plug into functions. Ask for clarification if a     │
│  user request is ambiguous.                                                                      │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ User ───────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  What is the weather going to be like in Santiago, Chile over the next x days?                   │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Assistant ──────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Could you please specify the number of days you would like the weather forecast for in          │
│  Santiago, Chile?                                                                                │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

In [52]:
messages.append({"role": "user", "content": "3 days"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
if len(assistant_message.tool_calls) > 0:
    messages.append({"role": "function", "content": f"{assistant_message.tool_calls[0].function}"})

pprint_messages(messages)

╭─ System ─────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Don't make assumptions about what values to plug into functions. Ask for clarification if a     │
│  user request is ambiguous.                                                                      │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ User ───────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  What is the weather going to be like in Santiago, Chile over the next x days?                   │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Assistant ──────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Could you please specify the number of days you would like the weather forecast for in          │
│  Santiago, Chile?                                                                                │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ User ───────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  3 days                                                                                          │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Function ───────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Function(arguments='{"location":"Santiago, Chile","format":"celsius","num_days":3}',            │
│  name='get_n_day_weather_forecast')                                                              │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

### Forcing the use of specific functions

In [53]:
# in this cell we force the model to use get_n_day_weather_forecast
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me a weather report for Buenos Aires, Argentina."})
chat_response = chat_completion_request(
    messages, tools=tools, tool_choice={"type": "function", "function": {"name": "get_n_day_weather_forecast"}}
)
assistant_message = chat_response.choices[0].message
if len(assistant_message.tool_calls) > 0:
    messages.append({"role": "function", "content": f"{assistant_message.tool_calls[0].function}"})

pprint_messages(messages)

╭─ System ─────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Don't make assumptions about what values to plug into functions. Ask for clarification if a     │
│  user request is ambiguous.                                                                      │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ User ───────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Give me a weather report for Buenos Aires, Argentina.                                           │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Function ───────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Function(arguments='{"location":"Buenos Aires, Argentina","format":"celsius","num_days":1}',    │
│  name='get_n_day_weather_forecast')                                                              │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

### Parallel Function Calling

In [54]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "What is the weather going to be like in Santiago and Buenos Aires over the next 4 days"})
chat_response = chat_completion_request(
    messages, tools=tools, model=GPT_MODEL
)

assistant_message = chat_response.choices[0].message
if len(assistant_message.tool_calls) > 0:
    for tool_call in assistant_message.tool_calls:
        messages.append({"role": "function", "content": f"{tool_call.function}"})

pprint_messages(messages)

╭─ System ─────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Don't make assumptions about what values to plug into functions. Ask for clarification if a     │
│  user request is ambiguous.                                                                      │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ User ───────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  What is the weather going to be like in Santiago and Buenos Aires over the next 4 days          │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Function ───────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Function(arguments='{"location": "Santiago, Chile", "format": "celsius", "num_days": 4}',       │
│  name='get_n_day_weather_forecast')                                                              │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Function ───────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Function(arguments='{"location": "Buenos Aires, Argentina", "format": "celsius", "num_days":    │
│  4}', name='get_n_day_weather_forecast')                                                         │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

## Function Calling Lifecycle

| ![fc-diagram](function-calling-diagram-resized.png) | 
|:--:| 
| *The lifecycle of a function call. Image from [OpenAI](https://platform.openai.com/docs/guides/function-calling/lifecycle).* |

### How to call functions with model generated arguments

In [55]:
import sqlite3

conn = sqlite3.connect("data/Chinook.db")
print("Opened database successfully")

Opened database successfully


In [56]:
def get_table_names(conn):
    """Return a list of table names."""
    table_names = []
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    for table in tables.fetchall():
        table_names.append(table[0])
    return table_names


def get_column_names(conn, table_name):
    """Return a list of column names."""
    column_names = []
    columns = conn.execute(f"PRAGMA table_info('{table_name}');").fetchall()
    for col in columns:
        column_names.append(col[1])
    return column_names


def get_database_info(conn):
    """Return a list of dicts containing the table name and columns for each table in the database."""
    table_dicts = []
    for table_name in get_table_names(conn):
        columns_names = get_column_names(conn, table_name)
        table_dicts.append({"table_name": table_name, "column_names": columns_names})
    return table_dicts


In [57]:
database_schema_dict = get_database_info(conn)
database_schema_string = "\n".join(
    [
        f"Table: {table['table_name']}\nColumns: {', '.join(table['column_names'])}"
        for table in database_schema_dict
    ]
)

In [60]:
print(database_schema_string)

Table: albums
Columns: AlbumId, Title, ArtistId
Table: sqlite_sequence
Columns: name, seq
Table: artists
Columns: ArtistId, Name
Table: customers
Columns: CustomerId, FirstName, LastName, Company, Address, City, State, Country, PostalCode, Phone, Fax, Email, SupportRepId
Table: employees
Columns: EmployeeId, LastName, FirstName, Title, ReportsTo, BirthDate, HireDate, Address, City, State, Country, PostalCode, Phone, Fax, Email
Table: genres
Columns: GenreId, Name
Table: invoices
Columns: InvoiceId, CustomerId, InvoiceDate, BillingAddress, BillingCity, BillingState, BillingCountry, BillingPostalCode, Total
Table: invoice_items
Columns: InvoiceLineId, InvoiceId, TrackId, UnitPrice, Quantity
Table: media_types
Columns: MediaTypeId, Name
Table: playlists
Columns: PlaylistId, Name
Table: playlist_track
Columns: PlaylistId, TrackId
Table: tracks
Columns: TrackId, Name, AlbumId, MediaTypeId, GenreId, Composer, Milliseconds, Bytes, UnitPrice
Table: sqlite_stat1
Columns: tbl, idx, stat


In [58]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "ask_database",
            "description": "Use this function to answer user questions about music. Input should be a fully formed SQL query.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": f"""
                                SQL query extracting info to answer the user's question.
                                SQL should be written using this database schema:
                                {database_schema_string}
                                The query should be returned in plain text, not in JSON.
                                """,
                    }
                },
                "required": ["query"],
            },
        }
    }
]

In [61]:
def ask_database(conn, query):
    """Function to query SQLite database with a provided SQL query."""
    try:
        results = str(conn.execute(query).fetchall())
    except Exception as e:
        results = f"query failed with error: {e}"
    return results

Steps to invoke a function call using Chat Completions API:

- **Step 1:** Prompt the model with content that may result in model selecting a tool to use. The description of the tools such as a function names and signature is defined in the 'Tools' list and passed to the model in API call. If selected, the function name and parameters are included in the response.
- **Step 2:** Check programmatically if model wanted to call a function. If true, proceed to step 3.
- **Step 3:** Extract the function name and parameters from response, call the function with parameters. Append the result to messages.
- **Step 4:** Invoke the chat completions API with the message list to get the response.

In [87]:
# Step #1: Prompt with content that may result in function call. In this case the model can identify the information requested by the user is potentially available in the database schema passed to the model in Tools description. 
messages = [{
    "role":"user", 
    "content": "What is the name of the album with the most tracks?"
}]

response = client.chat.completions.create(
    model='gpt-4o', 
    messages=messages, 
    tools= tools, 
    tool_choice="auto"
)

# Append the message to messages list
assistant_message = response.choices[0].message 
if len(assistant_message.tool_calls) > 0:
    for tool_call in assistant_message.tool_calls:
        messages.append({"role": "function", "content": f"{tool_call.function}"})

pprint_messages(messages)

╭─ User ───────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  What is the name of the album with the most tracks?                                             │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Function ───────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Function(arguments='{"query":"SELECT albums.Title, COUNT(tracks.TrackId) as TrackCount FROM     │
│  albums JOIN tracks ON albums.AlbumId = tracks.AlbumId GROUP BY albums.AlbumId ORDER BY          │
│  TrackCount DESC LIMIT 1;"}', name='ask_database')                                               │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

In [88]:
# Step 2: determine if the response from the model includes a tool call.   
# drop the last message from the list
messages.pop()
tool_calls = assistant_message.tool_calls
if tool_calls:
    # If true the model will return the name of the tool / function to call and the argument(s)
    tool_call_id = tool_calls[0].id
    tool_function_name = tool_calls[0].function.name
    tool_query_string = json.loads(tool_calls[0].function.arguments)['query']

    messages.append({
            "role": "assistant", 
            "tool_calls": tool_calls, 
            "content": f"{tool_calls[0].function}"
        })

    # Step 3: Call the function and retrieve results. Append the results to the messages list.      
    if tool_function_name == 'ask_database':
        results = ask_database(conn, tool_query_string)
        
        messages.append({
            "role": "tool", 
            "tool_call_id": tool_call_id, 
            "name": tool_function_name, 
            "content": results
        })
        
        # Step 4: Invoke the chat completions API with the function response appended to the messages list
        # Note that messages with role 'tool' must be a response to a preceding message with 'tool_calls'
        model_response_with_function_call = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        assistant_message = model_response_with_function_call.choices[0].message
        messages.append({"role": "assistant", "content": assistant_message.content})
    else: 
        print(f"Error: function {tool_function_name} does not exist")
else: 
    # Model did not identify a function to call, result can be returned to the user 
    messages.append({"role": "assistant", "content": assistant_message.content})

In [91]:
pprint_messages(messages)

╭─ User ───────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  What is the name of the album with the most tracks?                                             │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Assistant ──────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  Function(arguments='{"query":"SELECT albums.Title, COUNT(tracks.TrackId) as TrackCount FROM     │
│  albums JOIN tracks ON albums.AlbumId = tracks.AlbumId GROUP BY albums.AlbumId ORDER BY          │
│  TrackCount DESC LIMIT 1;"}', name='ask_database')                                               │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Tool ───────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  [('Greatest Hits', 57)]                                                                         │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Assistant ──────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                  │
│  The album with the most tracks is "Greatest Hits," which contains 57 tracks.                    │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯